1. Использовать dataset MovieLens (https://grouplens.org/datasets/movielens/latest/)


2. Построить рекомендации (регрессия, предсказываем оценку) на фичах:

 - TF-IDF на тегах и жанрах
 - Средние оценки (+ median, variance, etc.) пользователя и фильма
 
 
3. Оценить RMSE на тестовой выборке

Item to User, предсказать оценку пользователя
 - Соединить отдельные csv
 - Признак Жанр tfidf
 - Взять одно пользователя
 - Сделать train_test_split
 - Обучить регрессию
 - Померить RMSE

In [29]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score

In [30]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

Будем предсказывать рейтинг для пользователя **1**

In [31]:
df_user1 = ratings.merge(movies, on='movieId', how='left')[ratings['userId'] == 1]

In [32]:
df_user1.shape

(232, 6)

In [33]:
df_user1.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [34]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [35]:
movie_genres = [change_string(g) for g in df_user1.genres.values]

In [36]:
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(movie_genres)

In [37]:
df1 = pd.DataFrame(X_train_tfidf.toarray(), columns = tfidf_vectorizer.get_feature_names())
res = pd.concat([df_user1, df1], axis=1)

In [38]:
res.head()

,userId,movieId,rating,timestamp,title,genres,action,adventure,animation,children,...,fantasy,filmnoir,horror,musical,mystery,romance,scifi,thriller,war,western
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0.000000,0.35712,0.545481,0.481093,...,0.461419,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.842163,0.0,0.000000,0.0,0.0
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller,0.477274,0.00000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.596707,0.0,0.0
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.822407,0.000000,0.0,0.568899,0.0,0.0
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.700519,0.000000,0.0,0.484584,0.0,0.0


In [39]:
y = res['rating']
X = res.drop(['rating', 'title', 'genres', 'userId', 'movieId'], axis=1)

In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [41]:
X_train.shape

(162, 18)

Обучаем модель

In [42]:
from sklearn.linear_model import LinearRegression

In [43]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [44]:
y_pred = model.predict(X_test)

In [45]:
from sklearn import metrics

In [46]:
print(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

0.6903364720027393
